In [1]:
from math import sqrt
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pydataset import data
import statistics
import acquire
import prepare

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

import explore
import warnings
warnings.filterwarnings("ignore")

import graphviz
from graphviz import Graph

### Using the titanic data, in your classification-exercises repository, create a notebook, model.ipynb where you will do the following:

In [204]:
train, validate, test = prepare.prep_titanic_data(acquire.get_titanic_data())

In [205]:
train.head(3)

,survived,pclass,age,sibsp,parch,fare,alone,sex_male,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
455,1,3,29.0,0,0,7.8958,1,1,1,0,0
380,1,1,42.0,0,0,227.5250,1,0,1,0,0
492,0,1,55.0,0,0,30.5000,1,1,0,0,1


### What is your baseline prediction?
### What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode).

In [206]:
#train.survived.value_counts()

In [207]:
#train["baseline"] = 0

In [208]:
#(train.survived==train.baseline).mean()

Baseline Model is .616

### When you make those predictions, what is your accuracy? This is your baseline accuracy.
    61% and yes this is my basline accuracy assuming no survivers 

### Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [209]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534 entries, 455 to 496
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   survived                 534 non-null    int64  
 1   pclass                   534 non-null    int64  
 2   age                      534 non-null    float64
 3   sibsp                    534 non-null    int64  
 4   parch                    534 non-null    int64  
 5   fare                     534 non-null    float64
 6   alone                    534 non-null    int64  
 7   sex_male                 534 non-null    uint8  
 8   embark_town_Cherbourg    534 non-null    uint8  
 9   embark_town_Queenstown   534 non-null    uint8  
 10  embark_town_Southampton  534 non-null    uint8  
dtypes: float64(2), int64(5), uint8(4)
memory usage: 35.5 KB


In [210]:
# create X & y version of train, where y is a series with just the target variable and X are all the features. 

X_train = train.drop(columns=['survived'])
y_train = train.survived

X_validate = validate.drop(columns=['survived'])
y_validate = validate.survived

X_test = test.drop(columns=['survived'])
y_test = test.survived

In [211]:
clf = DecisionTreeClassifier(max_depth=3, random_state=123)

In [212]:
len(X_train.columns)

10

In [213]:
clf = clf.fit(X_train, y_train)

In [214]:
y_pred = clf.predict(X_train)
y_pred[0:5]

array([0, 1, 0, 0, 0])

In [215]:
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba[0:5]

array([[0.90410959, 0.09589041],
       [0.03092784, 0.96907216],
       [0.69105691, 0.30894309],
       [0.69105691, 0.30894309],
       [0.90410959, 0.09589041]])

### Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [216]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.83


In [217]:
confusion_matrix(y_train, y_pred)

array([[296,  33],
       [ 60, 145]])

In [218]:
y_train.value_counts()

0    329
1    205
Name: survived, dtype: int64

In [219]:
labels = sorted(y_train.unique())

pd.DataFrame(confusion_matrix(y_train, y_pred), index=labels, columns=labels)

,0,1
0,296,33
1,60,145


In [220]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.90      0.86       329
           1       0.81      0.71      0.76       205

    accuracy                           0.83       534
   macro avg       0.82      0.80      0.81       534
weighted avg       0.82      0.83      0.82       534



In [221]:
dot_data = export_graphviz(clf, feature_names= X_train.columns,class_names=['died','lived'], rounded=True, filled=True, out_file=None)
graph = graphviz.Source(dot_data) 

graph.render('titanic_decision_tree', view=True)

'titanic_decision_tree.pdf'

# Run through steps 2-4 using a different max_depth value.

In [222]:
def train_model(max_depth):
    clf = DecisionTreeClassifier(max_depth=max_depth, random_state=123)
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_train)
    y_pred_proba = clf.predict_proba(X_train)
    accuracy = clf.score(X_train, y_train)
    #print(classification_report(y_train, y_pred))
    return accuracy

def validate_model(max_depth):
    clf = DecisionTreeClassifier(max_depth=max_depth, random_state=123)
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_validate)
    y_pred_proba = clf.predict_proba(X_validate)
    accuracy = clf.score(X_validate, y_validate)
    return accuracy
    #print(classification_report(y_validate, y_pred))

In [223]:
# for depth in range(1,7):
#     print(f'Accuracy of Decision Tree classifier in depth: {depth}: train = {round(train_model(depth), 3)} validate = {round(validate_model(depth), 3)}')

### Which model performs better on your in-sample data?
    Max Depth 6 worked better in my sample data

### Which model performs best on your out-of-sample data, the validate set?
    Max Depth 2 was the closest accuracy to my train set

# Random Forest Exercises

### Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.

In [224]:
rf = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=1,
                            n_estimators=100,
                            max_depth=10, 
                            random_state=123)

In [225]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=123)

### Evaluate your results using the model score, confusion matrix, and classification report.

In [226]:
y_pred = rf.predict(X_train)
y_pred[0:5]

array([0, 1, 0, 1, 0])

In [227]:
y_pred_proba = rf.predict_proba(X_train)
y_pred_proba

array([[0.52939928, 0.47060072],
       [0.02      , 0.98      ],
       [0.79760646, 0.20239354],
       ...,
       [0.8561691 , 0.1438309 ],
       [0.77788441, 0.22211559],
       [0.04142857, 0.95857143]])

In [228]:
print('Accuracy of random forest classifier on training set: {:.2f}'
     .format(rf.score(X_train, y_train)))

Accuracy of random forest classifier on training set: 0.97


In [229]:
print(confusion_matrix(y_train, y_pred))

[[329   0]
 [ 17 188]]


In [230]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       329
           1       1.00      0.92      0.96       205

    accuracy                           0.97       534
   macro avg       0.98      0.96      0.97       534
weighted avg       0.97      0.97      0.97       534



In [231]:
print('Accuracy of random forest classifier on test set: {:.2f}'
     .format(rf.score(X_validate, y_validate)))

Accuracy of random forest classifier on test set: 0.80


In [232]:
train.columns

Index(['survived', 'pclass', 'age', 'sibsp', 'parch', 'fare', 'alone',
       'sex_male', 'embark_town_Cherbourg', 'embark_town_Queenstown',
       'embark_town_Southampton'],
      dtype='object')

In [233]:
rf.feature_importances_

array([0.0954103 , 0.21026042, 0.0494635 , 0.03437258, 0.24743484,
       0.02083179, 0.28993623, 0.0162347 , 0.01552962, 0.02052602])

1. fare
2. sex_male
3. pclass

In [234]:
def forest_train(leaf, depth):
    rf = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=leaf,
                            n_estimators=100,
                            max_depth=depth, 
                            random_state=123)
    rf.fit(X_train, y_train)
    accuracy = rf.score(X_train, y_train)
    return round(accuracy, 3)

def forest_validate(leaf, depth):
    rf = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=leaf,
                            n_estimators=100,
                            max_depth=depth, 
                            random_state=123)
    rf.fit(X_train, y_train)
    accuracy = rf.score(X_validate, y_validate)
    return round(accuracy, 3)

# What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

In [250]:
# Searching for best Train
for leaf in range(1,11):
    for depth in range(1,11):
        f_train = forest_train(leaf, depth)
        f_validate = forest_validate(leaf, depth) # i didn't reference f_validate in this loop
        difference = abs(f_train - f_validate) # didn't use the difference in this loop
        score = f_train
        if (leaf == 1) & (depth == 1):
            best_difference = difference
            best_leaf = leaf
            best_depth = depth
            best_score = score
            
        else:
            if score >= best_score:
                best_leaf = leaf
                best_depth = depth
                best_difference = difference
                best_score = score
                print(f"Best Train Accuracy:{best_score}, Leaf: {best_leaf}, Depth:{best_depth}")

Best Train Accuracy:0.8, Leaf: 1, Depth:2
Best Train Accuracy:0.835, Leaf: 1, Depth:3
Best Train Accuracy:0.845, Leaf: 1, Depth:4
Best Train Accuracy:0.869, Leaf: 1, Depth:5
Best Train Accuracy:0.893, Leaf: 1, Depth:6
Best Train Accuracy:0.927, Leaf: 1, Depth:7
Best Train Accuracy:0.942, Leaf: 1, Depth:8
Best Train Accuracy:0.959, Leaf: 1, Depth:9
Best Train Accuracy:0.968, Leaf: 1, Depth:10


# After making a few models, which one has the best performance (or closest metrics) on both train and validate?

In [251]:
# Searching for best Validate 


for leaf in range(1,11):
    for depth in range(1,11):
        f_train = forest_train(leaf, depth)
        f_validate = forest_validate(leaf, depth)
        difference = abs(f_train - f_validate)
        score = f_validate
        
        if (leaf == 1) & (depth == 1):
            best_difference = difference
            best_leaf = leaf
            best_depth = depth
            best_score = score
            
        else:
            if score >= best_score:
                best_leaf = leaf
                best_depth = depth
                best_difference = difference
                best_score = score
                print(f"Best Validate Accuracy:{best_score}, Leaf: {best_leaf}, Depth:{best_depth}, threshold:{best_difference}")

Best Validate Accuracy:0.787, Leaf: 1, Depth:2, threshold:0.013000000000000012
Best Validate Accuracy:0.809, Leaf: 1, Depth:3, threshold:0.025999999999999912
Best Validate Accuracy:0.815, Leaf: 1, Depth:4, threshold:0.030000000000000027
Best Validate Accuracy:0.826, Leaf: 1, Depth:5, threshold:0.04300000000000004
Best Validate Accuracy:0.826, Leaf: 2, Depth:5, threshold:0.04500000000000004
Best Validate Accuracy:0.831, Leaf: 3, Depth:5, threshold:0.029000000000000026


    Best Validate Model was the one with 3 Leaf and Depth of 5 with an accuracy of .831 and a threshold of .029

# KNN

### Fit a K-Nearest Neighbors classifier to your training sample and transform (i.e. make predictions on the training sample)

In [34]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')

In [35]:
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [36]:
y_pred = knn.predict(X_train)

In [37]:
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))

Accuracy of KNN classifier on training set: 0.79


### Evaluate your results using the model score, confusion matrix, and classification report.

    Model Score

In [38]:
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))

Accuracy of KNN classifier on training set: 0.79


    Confusion Matrix

In [39]:
print(confusion_matrix(y_train, y_pred))

[[278  51]
 [ 60 145]]


    Classification Report

In [40]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.84      0.83       329
           1       0.74      0.71      0.72       205

    accuracy                           0.79       534
   macro avg       0.78      0.78      0.78       534
weighted avg       0.79      0.79      0.79       534



### Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [41]:
report = classification_report(y_train, y_pred, output_dict=True)
print(pd.DataFrame(report))
print()

                    0           1  accuracy   macro avg  weighted avg
precision    0.822485    0.739796  0.792135    0.781141      0.790741
recall       0.844985    0.707317  0.792135    0.776151      0.792135
f1-score     0.833583    0.723192  0.792135    0.778388      0.791205
support    329.000000  205.000000  0.792135  534.000000    534.000000



### Run through steps 2-4 setting k to 10

In [42]:
def knn(k):
    knn = KNeighborsClassifier(n_neighbors=k, weights='uniform')
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_train)
    report = classification_report(y_train, y_pred, output_dict=True)
    print(f"K as {k}")
    print(pd.DataFrame(report))

In [43]:
knn(10)

K as 10
                    0           1  accuracy   macro avg  weighted avg
precision    0.745636    0.774436  0.752809    0.760036      0.756692
recall       0.908815    0.502439  0.752809    0.705627      0.752809
f1-score     0.819178    0.609467  0.752809    0.714323      0.738671
support    329.000000  205.000000  0.752809  534.000000    534.000000


### Run through setps 2-4 setting k to 20

In [44]:
knn(20)

K as 20
                    0           1  accuracy   macro avg  weighted avg
precision    0.715663    0.731092  0.719101    0.723378      0.721586
recall       0.902736    0.424390  0.719101    0.663563      0.719101
f1-score     0.798387    0.537037  0.719101    0.667712      0.698056
support    329.000000  205.000000  0.719101  534.000000    534.000000


### What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

    Accuracy is going down as you have more neighbors in the in-sample data. Probably because the calculated distance is not
    as seperated as one might think.

### Which model performs best on our out-of-sample data from validate?

In [193]:
def knn_validate(k):
    knn = KNeighborsClassifier(n_neighbors=k, weights='uniform')
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_validate)
    report = classification_report(y_validate, y_pred, output_dict=True)
    print(f"K as {k}")
    print(pd.DataFrame(report))

In [194]:
knn_validate(5)

K as 5
                    0          1  accuracy   macro avg  weighted avg
precision    0.734513   0.584615  0.679775    0.659564      0.677249
recall       0.754545   0.558824  0.679775    0.656684      0.679775
f1-score     0.744395   0.571429  0.679775    0.657912      0.678318
support    110.000000  68.000000  0.679775  178.000000    178.000000


In [195]:
knn_validate(10)

K as 10
                    0          1  accuracy   macro avg  weighted avg
precision    0.732283   0.666667  0.713483    0.699475      0.707216
recall       0.845455   0.500000  0.713483    0.672727      0.713483
f1-score     0.784810   0.571429  0.713483    0.678119      0.703294
support    110.000000  68.000000  0.713483  178.000000    178.000000


In [196]:
knn_validate(20)

K as 20
                    0          1  accuracy   macro avg  weighted avg
precision    0.752000   0.698113  0.735955    0.725057      0.731414
recall       0.854545   0.544118  0.735955    0.699332      0.735955
f1-score     0.800000   0.611570  0.735955    0.705785      0.728016
support    110.000000  68.000000  0.735955  178.000000    178.000000


    Knn where k=20 preformed the best in Validate at .735

---

# <b>Logistic Regression</b>

### For all of the models you create, choose a threshold that optimizes for accuracy.

In [170]:
threshold = .05

### Do your work for these exercises in either a notebook or a python script named model within your classification-exercises repository. Add, commit, and push your work.

    Baseline

In [133]:
train, validate, test = prepare.prep_titanic_data(acquire.get_titanic_data())

In [134]:
train['survived'].value_counts()

0    329
1    205
Name: survived, dtype: int64

In [135]:
train['baseline'] = 0

In [136]:
train.head()

,survived,pclass,age,sibsp,parch,fare,alone,sex_male,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton,baseline
455,1,3,29.000000,0,0,7.8958,1,1,1,0,0,0
380,1,1,42.000000,0,0,227.5250,1,0,1,0,0,0
492,0,1,55.000000,0,0,30.5000,1,1,0,0,1,0
55,1,1,29.393566,0,0,35.5000,1,1,0,0,1,0
243,0,3,22.000000,0,0,7.1250,1,1,0,0,1,0


In [137]:
(train['survived']==train['baseline']).mean()

0.6161048689138576

### Create a model that includes age in addition to fare and pclass. Does this model perform better than your baseline?

### Model 1

In [138]:
train, validate, test = prepare.prep_titanic_data(acquire.get_titanic_data())

In [139]:
X_train = train[['age', 'fare', 'pclass']]
y_train = train.survived

logit = LogisticRegression(random_state=123)
logit.fit(X_train, y_train)

y_pred = logit.predict(X_train)

model1_train_accuracy = logit.score(X_train, y_train)
print('Accuracy of Logistic Regression classifier on training set "age", "fare", "pclass": {:.2f}'
     .format(logit.score(X_train, y_train)))

Accuracy of Logistic Regression classifier on training set "age", "fare", "pclass": 0.71


    Does not beat the baseline

### Include sex in your model as well. Note that you'll need to encode or create a dummy variable of this feature before including it in a model.

### Model 2

In [140]:
X_train = train[['age', 'fare', 'pclass', 'sex_male']]
y_train = train.survived

logit = LogisticRegression(random_state=123)
logit.fit(X_train, y_train)

y_pred = logit.predict(X_train)

model2_train_accuracy = logit.score(X_train, y_train)
print('Accuracy of Logistic Regression classifier on training set "age", "fare", "pclass", "sex_male": {:.2f}'
     .format(logit.score(X_train, y_train)))

Accuracy of Logistic Regression classifier on training set "age", "fare", "pclass", "sex_male": 0.81


### Try out other combinations of features and models.

### Model 3

    Here I included all the features

In [141]:
train, validate, test = prepare.prep_titanic_data(acquire.get_titanic_data())

In [142]:
X_train = train.drop(columns=['survived'])
y_train = train.survived

X_validate = validate.drop(columns=['survived'])
y_validate = validate.survived

X_test = test.drop(columns=['survived'])
y_test = test.survived

In [143]:
logit = LogisticRegression(random_state=123)
logit.fit(X_train, y_train)

LogisticRegression(random_state=123)

In [144]:
y_pred = logit.predict(X_train)

In [145]:
y_pred_proba = logit.predict_proba(X_train)

In [146]:
model3_train_accuracy = logit.score(X_train, y_train)
print('Accuracy of Logistic Regression classifier on training set on All Features: {:.2f}'
     .format(logit.score(X_train, y_train)))

Accuracy of Logistic Regression classifier on training set on All Features: 0.80


### Model 4

    All the features and Changed C to .1

In [147]:
X_train = train.drop(columns=['survived'])
y_train = train.survived

X_validate = validate.drop(columns=['survived'])
y_validate = validate.survived

X_test = test.drop(columns=['survived'])
y_test = test.survived

logit = LogisticRegression(C=.1, random_state=123) # <--- C to .5 here
logit.fit(X_train, y_train)
y_pred = logit.predict(X_train)

model4_train_accuracy = logit.score(X_train, y_train)
print('Accuracy of Logistic Regression classifier on training set on All Features: {:.2f}'
     .format(logit.score(X_train, y_train)))

Accuracy of Logistic Regression classifier on training set on All Features: 0.82


### Use you best 3 models to predict and evaluate on your validate sample.

### Model 1 Using the features 'age', 'fare', 'pclass'

In [252]:
train[['age', 'fare', 'pclass']]

,age,fare,pclass
455,29.000000,7.8958,3
380,42.000000,227.5250,1
492,55.000000,30.5000,1
55,29.393566,35.5000,1
243,22.000000,7.1250,3
...,...,...,...
695,52.000000,13.5000,2
128,29.393566,22.3583,3
636,32.000000,7.9250,3
222,51.000000,8.0500,3


In [182]:
X_train = train[['age', 'fare', 'pclass']]
y_train = train.survived

X_validate = validate[['age', 'fare', 'pclass']]
y_validate = validate.survived

logit = LogisticRegression(random_state=123)
logit.fit(X_train, y_train)

y_pred = logit.predict(X_validate)
model1_validate_accuracy = logit.score(X_validate, y_validate)
difference = abs(model1_validate_accuracy-model1_train_accuracy)

print(f"Model 1 features ('age', 'fare', 'pclass')\n\nTrain accuracy = {model1_train_accuracy}\nvalidate accuracy = {model1_validate_accuracy}\nDifferece: {difference}")
print(f"Meets threshold({threshold}): requirement: {threshold>difference}")

Model 1 features ('age', 'fare', 'pclass')

Train accuracy = 0.7059925093632958
validate accuracy = 0.6966292134831461
Differece: 0.009363295880149725
Meets threshold(0.05): requirement: True


### Model 2  Using the features 'age', 'fare', 'pclass', 'sex_male'

In [178]:
X_train = train[['age', 'fare', 'pclass', 'sex_male']]
y_train = train.survived

X_validate = validate[['age', 'fare', 'pclass', 'sex_male']]
y_validate = validate.survived

logit = LogisticRegression(random_state=123)
logit.fit(X_train, y_train)

y_pred = logit.predict(X_validate)
model2_validate_accuracy = logit.score(X_validate, y_validate)
difference = abs(model2_validate_accuracy-model2_train_accuracy)

print(f"Model 2 features ('age', 'fare', 'pclass', 'sex_male')\n\nTrain accuracy = {model2_train_accuracy}\nvalidate accuracy = {model2_validate_accuracy}\nDifferece: {difference}")
print(f"Meets threshold({threshold}): requirement: {threshold>difference}")

Model 2 features ('age', 'fare', 'pclass', 'sex_male')

Train accuracy = 0.8052434456928839
validate accuracy = 0.7808988764044944
Differece: 0.02434456928838946
Meets threshold(0.05): requirement: True


### Model 3  Using ALL Features

In [175]:
X_train= train.drop(columns=['survived'])
y_train = train.survived

X_validate = validate.drop(columns=['survived'])
y_validate = validate.survived


logit = LogisticRegression(random_state=123)
logit.fit(X_train, y_train)
y_pred = logit.predict(X_validate)

model3_validate_accuracy = logit.score(X_validate, y_validate)
difference = abs(model3_validate_accuracy-model3_train_accuracy)

print(f"Model 3 features ('ALL Features')\n\nTrain accuracy = {model3_train_accuracy}\nvalidate accuracy = {model3_validate_accuracy}\nDifferece: {difference}")
print(f"Meets threshold({threshold}): requirement: {threshold>difference}")

Model 3 features ('ALL Features')

Train accuracy = 0.8014981273408239
validate accuracy = 0.7921348314606742
Differece: 0.009363295880149725
Meets threshold(0.05): requirement: True


### Model 4 Using ALL Features and Setting C to .1

In [184]:
X_train = train.drop(columns=['survived'])
y_train = train.survived

X_validate = validate.drop(columns=['survived'])
y_validate = validate.survived

logit = LogisticRegression(C=.1, random_state=123) # < -- C=.1
logit.fit(X_train, y_train)
y_pred = logit.predict(X_validate)

model4_validate_accuracy = logit.score(X_validate, y_validate)
difference = abs(model4_validate_accuracy-model4_train_accuracy)

print(f"Model 4 features ('ALL Features and C=.1')\n\nTrain accuracy = {model4_train_accuracy}\nvalidate accuracy = {model4_validate_accuracy}\nDifferece: {difference}")
print(f"Meets threshold({threshold}): requirement: {threshold>difference}")

Model 4 features ('ALL Features and C=.1')

Train accuracy = 0.8183520599250936
validate accuracy = 0.7921348314606742
Differece: 0.02621722846441943
Meets threshold(0.05): requirement: True


### Choose you best model from the validation performation, and evaluate it on the test dataset. How do the performance metrics compare to validate? to train?

    My best Model was Model 3 (ALl Features) with a .792 Accuracy and lowest Threshold of .009

# Running Test Model

In [192]:
X_train= train.drop(columns=['survived'])
y_train = train.survived

X_test = test.drop(columns=['survived'])
y_test = test.survived


logit = LogisticRegression(random_state=123)
logit.fit(X_train, y_train)
y_pred = logit.predict(X_test)

model3_test_accuracy = logit.score(X_test, y_test)
difference_train = abs(model3_test_accuracy-model3_train_accuracy)
difference_validate = abs(model3_test_accuracy-model3_validate_accuracy)


print(f"Train Accuracy: {model3_train_accuracy}")
print(f"Validate Accuracy: {model3_validate_accuracy}")
print(f"Test Accuracy: {model3_test_accuracy}\n")

print(f"Test-Train: {difference_train}")
print(f"Test-Validate: {difference_validate}")


Train Accuracy: 0.8014981273408239
Validate Accuracy: 0.7921348314606742
Test Accuracy: 0.8100558659217877

Test-Train: 0.008557738580963759
Test-Validate: 0.017921034461113483
